## Deliverable 2. Create a Customer Travel Destinations Map.

In [17]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
import os
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df_to_load = os.path.join("..","Weather_Database","WeatherPy_Database.csv")
city_data_df = pd.read_csv(city_data_df_to_load)
city_data_df.head(11)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Rikitea,PF,-23.1203,-134.9692,78.46,72,83,11.50
1,1,Maneadero,MX,31.7167,-116.5667,78.13,22,2,7.65
2,2,New Norfolk,AU,-42.7826,147.0587,60.93,73,93,1.59
3,3,Bathsheba,BB,13.2167,-59.5167,84.81,61,75,17.27
4,4,Cidreira,BR,-30.1811,-50.2056,69.94,91,100,4.12
5,5,Bluff,NZ,-46.6000,168.3333,53.80,79,9,4.00
6,6,Upernavik,GL,72.7868,-56.1549,-6.74,97,100,13.49
7,7,Mahebourg,MU,-20.4081,57.7000,81.25,89,40,8.05
8,8,Tuktoyaktuk,CA,69.4541,-133.0374,-32.80,68,75,3.44
9,9,Poum,NC,-20.2333,164.0167,81.48,77,95,12.95


In [11]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the Maximum temperature you would like for your trip? "))

In [12]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(11)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
3,3,Bathsheba,BB,13.2167,-59.5167,84.81,61,75,17.27
7,7,Mahebourg,MU,-20.4081,57.7000,81.25,89,40,8.05
9,9,Poum,NC,-20.2333,164.0167,81.48,77,95,12.95
17,17,Grand Gaube,MU,-20.0064,57.6608,81.37,79,89,5.99
24,24,Arraial Do Cabo,BR,-22.9661,-42.0278,80.56,78,40,14.97
27,27,Puerto Ayora,EC,-0.7393,-90.3518,80.56,88,91,4.00
39,39,Acari,PE,-15.4311,-74.6158,80.62,57,39,10.22
50,50,Bambous Virieux,MU,-20.3428,57.7575,81.21,89,40,8.05
53,53,Avarua,CK,-21.2078,-159.7750,82.45,74,20,2.30
55,55,San Felipe,VE,10.3399,-68.7425,80.80,60,95,10.71


In [16]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID       0
City          0
Country       0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
dtype: int64

In [21]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# No Empty Spaces!!

In [23]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Cloudiness", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(11)

,City,Country,Max Temp,Cloudiness,Lat,Lng,Hotel Name
3,Bathsheba,BB,84.81,75,13.2167,-59.5167,
7,Mahebourg,MU,81.25,40,-20.4081,57.7000,
9,Poum,NC,81.48,95,-20.2333,164.0167,
17,Grand Gaube,MU,81.37,89,-20.0064,57.6608,
24,Arraial Do Cabo,BR,80.56,40,-22.9661,-42.0278,
27,Puerto Ayora,EC,80.56,91,-0.7393,-90.3518,
39,Acari,PE,80.62,39,-15.4311,-74.6158,
50,Bambous Virieux,MU,81.21,40,-20.3428,57.7575,
53,Avarua,CK,82.45,20,-21.2078,-159.7750,
55,San Felipe,VE,80.80,95,10.3399,-68.7425,


In [25]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [28]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.head()

,City,Country,Max Temp,Cloudiness,Lat,Lng,Hotel Name
3,Bathsheba,BB,84.81,75,13.2167,-59.5167,Atlantis Hotel
7,Mahebourg,MU,81.25,40,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
9,Poum,NC,81.48,95,-20.2333,164.0167,
17,Grand Gaube,MU,81.37,89,-20.0064,57.6608,Veranda Paul et Virginie Hotel & Spa
24,Arraial Do Cabo,BR,80.56,40,-22.9661,-42.0278,Pousada Porto Praia


In [32]:
hotel_df.replace('',np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(11)

,City,Country,Max Temp,Cloudiness,Lat,Lng,Hotel Name
3,Bathsheba,BB,84.81,75,13.2167,-59.5167,Atlantis Hotel
7,Mahebourg,MU,81.25,40,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
17,Grand Gaube,MU,81.37,89,-20.0064,57.6608,Veranda Paul et Virginie Hotel & Spa
24,Arraial Do Cabo,BR,80.56,40,-22.9661,-42.0278,Pousada Porto Praia
27,Puerto Ayora,EC,80.56,91,-0.7393,-90.3518,Finch Bay Galapagos Hotel
39,Acari,PE,80.62,39,-15.4311,-74.6158,HOSPEDAJE XIMENA
50,Bambous Virieux,MU,81.21,40,-20.3428,57.7575,Casa Tia Villa
53,Avarua,CK,82.45,20,-21.2078,-159.7750,Paradise Inn
55,San Felipe,VE,80.80,95,10.3399,-68.7425,Posada Llovizna Rodríguez
59,Isangel,VU,80.71,40,-19.5500,169.2667,Tanna Lodge


In [33]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [40]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [44]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))